# Mice data exercise

Perform the following filtering analysis:

- **1. Missingness and filtering, both at the variant- and individual-level.**
- **2. Minor Allele Frequency filtering.**
- **3. Hardy-Weinberg Equilibrium.**
- **4. Heterozygosity rate.**

<img src="../Images/bash.png" alt="bash" width="40">  Switch to the Bash kernel.


In [ ]:
ln -sf ../../Data
ln -sf ../Results

# **1. Filtering excessive missing values.** 

First of all, we begin by producing the sample-wise and variant-wise missing reports in the format `.imiss` and `.lmiss`. Note that sex will not be considered at all by PLINK as mentioned in the warning.

In [ ]:
plink --bfile Data/mice --missing --out Results/GWAS3/mice --silent

<img src="../Images/R.png" alt="R" width="40">  Switch to the R-GWAS kernel.
 
Now we look at the histograms. The only thing needed to change in the code, compared to the tutorial, is the name of the two missingness reports.

In [ ]:
library(ggplot2)
options(repr.plot.width = 9, repr.plot.height = 4)

# Read data into R 
indmiss <- read.table(file="Results/GWAS3/mice.imiss", header=TRUE)
snpmiss <- read.table(file="Results/GWAS3/mice.lmiss", header=TRUE)

#imiss histogram
hist.imiss <- ggplot(indmiss, aes(x=indmiss[,6])) +
  geom_histogram(binwidth = 0.001, col = "black", fill="tomato") + 
  labs(title = "Frequency of missingness rates in individuals") + 
  xlab("Frequency") + 
  ylab("Missingness in Individuals") + 
  theme_bw()+
  theme(axis.title=element_text(size=13), axis.text=element_text(size=13), plot.title=element_text(size=15))

#lmiss histogram
hist.lmiss <- ggplot(snpmiss, aes(x=snpmiss[,5])) +
  geom_histogram(binwidth = 0.005, col = "black", fill="tomato") + 
  labs(title = "Frequency of missingness rates in SNPs") + 
  xlab("Frequency") + 
  ylab("Missingness in SNPs") + 
  theme_bw()+
  theme(axis.title=element_text(size=13), axis.text=element_text(size=13), plot.title=element_text(size=15))

#show histograms
show(hist.imiss)
show(hist.lmiss)

The histograms do not show any high missingness for the variants (second plot), while there are a few individuals that we should filter out. Below, we filter only individuals with missingness above 2%, without choosing any variant filter. The new `bed, bim, fam` files have the prefix `mice_miss`

In [ ]:
plink --bfile Data/mice --mind .02 --make-bed --out Results/GWAS3/mice_miss --silent

# **2. Minimum Allele Frequency**. 
Let's calculate the MAF values using `--freq`: 

In [ ]:
# Generate a plot of the MAF distribution.
plink --bfile Results/GWAS3/mice_miss --freq --out Results/GWAS3/mice_miss --silent

<img src="../Images/R.png" alt="R" width="40">  Switch to the R-GWAS kernel.

Let's plot the MAF histogram using `R`. The second plot, is a zoom-in version with MAF up to 0.1 to observe the low MAF values. 

In [ ]:
#| label: fig-MAF
#| fig-cap: "MAF plot for the autosomal SNPs only. Note the zoomed interval (0, 0.1)"

library(ggplot2)
options(repr.plot.width = 9, repr.plot.height = 4)

# Read data into R 
maf_freq <- read.table("Results/GWAS3/mice_miss.frq", header =TRUE, as.is=T)

# maf_freq histogram
hist.maf <- ggplot(maf_freq, aes(x=maf_freq[,5])) +
  geom_histogram(col = "black", fill="tomato", bins=50) + 
  labs(title = "MAF distribution") + 
  xlab("MAF") + 
  ylab("Frequency") +
  theme_bw()+
  theme(axis.title=element_text(size=13), axis.text=element_text(size=13), plot.title=element_text(size=15))

# zoom-in into X-axis
hist.maf.zoom <- ggplot(maf_freq, aes(x=maf_freq[,5])) +
  geom_histogram(col = "black", fill="tomato", bins = 10) + 
  labs(title = "Zoom into MAF distribution up to value 0.1") + 
  xlab("MAF") + 
  ylab("Frequency") +
  xlim(-.01, 0.1) +
  theme_bw()+
  theme(axis.title=element_text(size=13), axis.text=element_text(size=13), plot.title=element_text(size=15))


show(hist.maf)
show(hist.maf.zoom)

There are two minor peaks in the histograms above, one at 0.05 and 0.1. Choosing 0.1 would remove over a hundred extra SNPs, while 0.05 should remove around less than 200. We can for example choose 0.05, but 0.1 is acceptable as well as it is another threshold often applied in literature.

In [ ]:
# Remove SNPs with a low MAF frequency.
plink --bfile Results/GWAS3/mice_miss --maf 0.05 --make-bed --out Results/GWAS3/mice_miss_maf --silent

# **3. Hardy Weinberg Equilibrium**. 

Here we need again to apply the option `hardy` to calculate observed and expected heterozigous frequencies and p-values for the binomial test. Note how column `TEST` considers all individuals ( `ALL(NP)` stands for `ALL Not Pruned`).

In [ ]:
plink --bfile Results/GWAS3/mice_miss_maf --hardy --out Results/GWAS3/mice_miss_maf --silent

awk '{ if ($9 <0.00001) print $0 }' Results/GWAS3/mice_miss_maf.hwe > Results/GWAS3/mice_miss_maf.deviating.hwe

In [ ]:
head Results/GWAS3/mice_miss_maf.hwe 

In [ ]:
wc -l Results/GWAS3/mice_miss_maf.deviating.hwe

There are 355 variants to remove with a p-value below 0.00001

<img src="../Images/R.png" alt="R" width="40">  Switch to the R-GWAS kernel.

Now we create histograms to look at the p-values from the tables above. We do not need to use the case-control renaming and plotting categories.

In [ ]:
suppressMessages(suppressWarnings(library(dplyr)))

# Read data into R using dplyr library 
hwe <- read.table(file="Results/GWAS3/mice_miss_maf.hwe", header=TRUE)
hwe_zoom <- read.table(file="Results/GWAS3/mice_miss_maf.deviating.hwe", header=FALSE)
colnames(hwe_zoom) <- colnames(hwe)

Here, we print the first rows of the two generated tables

In [ ]:
#| label: fig-HWEtables
#| fig-cap: "HWE tables from PLINK"

head(hwe)
head(hwe_zoom)

Now, we can plot the histograms

In [ ]:
#| label: fig-HWE
#| fig-cap: "HWE p-values calculated with PLINK and zoom for the SNPs deviating from HWE."
suppressMessages(suppressWarnings(library(ggplot2)))
options(repr.plot.width = 9, repr.plot.height = 4)

# maf_freq histogram
hist.hwe <- ggplot(hwe, aes(x=hwe[,9])) +
  geom_histogram(col = "black", fill="tomato", bins=50) + 
  labs(title = "HWE distribution") + 
  xlab("HWE p-value") + 
  ylab("Frequency") +
  theme_bw()+
  theme(axis.title=element_text(size=13), axis.text=element_text(size=13), plot.title=element_text(size=15))

# maf_freq histogram
hist.hwe_below_threshold <- ggplot(hwe_zoom, aes(x=hwe_zoom[,9])) +
  geom_histogram(binwidth = 0.0000015, col = "black", fill="tomato") + 
  labs(title = "HWE distribution for strongly deviating SNPs only") + 
  xlab("HWE p-value") + 
  ylab("Frequency") +
  theme_bw()+
  theme(axis.title=element_text(size=13), axis.text=element_text(size=13), plot.title=element_text(size=15))

show(hist.hwe)
show(hist.hwe_below_threshold)

We choose the threshold 1e-5 and create the new filtered data files:

In [ ]:
plink --bfile Results/GWAS3/mice_miss_maf --hwe 1e-5 --make-bed --out Results/GWAS3/mice_miss_maf_hwe --silent

# **4. Heterozigosity rate.** 
Now we look at genotype calls at each individual's level. First, we create a list of SNPs for which there is lower correlation, so to avoid linkage disequilibrium.

In [ ]:
plink --bfile Results/GWAS3/mice_miss_maf_hwe --indep-pairwise 50 5 0.2 --out Results/GWAS3/mice_indepSNP --silent

With this pruned list, we measure the heterozygosity rates of the individuals in the remaining independent SNPs.

In [ ]:
plink --bfile Results/GWAS3/mice_miss_maf_hwe \
    --extract Results/GWAS3/mice_indepSNP.prune.in \
    --het \
    --out Results/GWAS3/mice_R_check \
    --silent

How does the table with the SNPs look like?

In [ ]:
head -5 Results/GWAS3/mice_R_check.het

<img src="../Images/R.png" alt="R" width="40">  Switch to the R-GWAS kernel.

We are going to calculate by hand the heterozygosity rates so we can plot them (not by default in the table). The formula for the rates is $N(NM)-O(HOM)/N(NM)$

In [ ]:
#| label: fig-HET
#| fig-cap: "Barplot of heterozigosity rates."

library(ggplot2)
options(repr.plot.width = 9, repr.plot.height = 4)

het <- read.table("Results/GWAS3/mice_R_check.het", head=TRUE)

het$HET_RATE = (het$"N.NM." - het$"O.HOM.")/het$"N.NM."

# plink.imiss histogram
hist.het <- ggplot(het, aes(x=HET_RATE)) +
  geom_histogram(binwidth = 0.01, col = "black", fill="tomato") + 
  labs(title = "Heterozygosity Rates") + 
  xlab("Heterozygosity Rate") + 
  ylab("Frequency") + 
  theme_bw()


show(hist.het)

We use the same filter for all individuals as applied to the other dataset (heterozigosity rate 3 standard deviations away from the mean)

In [ ]:
suppressMessages(suppressWarnings(library(dplyr)))

het <- read.table("Results/GWAS3/mice_R_check.het", head=TRUE)
het$HET_RATE = (het$"N.NM." - het$"O.HOM.")/het$"N.NM."

het_fail <- subset(het, (het$HET_RATE < mean(het$HET_RATE)-3*sd(het$HET_RATE)) | 
                    (het$HET_RATE > mean(het$HET_RATE)+3*sd(het$HET_RATE)))

het_fail$HET_DST <- (het_fail$HET_RATE-mean(het$HET_RATE))/sd(het$HET_RATE);

write.table(het_fail, "Results/GWAS3/mice-fail-het-qc.txt", row.names=FALSE, quote=FALSE)

The resulting table contains a few outlying individuals:

In [ ]:
wc -l Results/GWAS3/mice-fail-het-qc.txt

We use `awk` to print out the first two columns in the file `het-fail-ind.txt` and use the option `--remove` to filter out the individuals with PLINK.

In [ ]:
awk '{print$1, $2}' Results/GWAS3/mice-fail-het-qc.txt > Results/GWAS3/mice-het-fail-ind.txt

plink --bfile Results/GWAS3/mice_miss_maf_hwe \
    --remove Results/GWAS3/mice-het-fail-ind.txt \
    --make-bed \
    --out Results/GWAS3/mice_QCA \
    --silent

We can count again samples and variants

In [ ]:
wc -l Results/GWAS3/mice_QCA.fam
wc -l Results/GWAS3/mice_QCA.bim